The goal of this notebook is to walk through an example use-case for the reweighted loss function. My goal is to give sufficient background information that those unfamilar with NLI can still follow along and understand the rationale for and results of changing the loss function. The work in this notebook was originally done for a final paper for a graduate class in NLP, but I wanted to repackage the content of the paper in a more accessable format, resulting in this notebook.

This notebook does not contain much code. If you would rather just see the huggingface and pytorch code used to train and evaluate the model see './python/run.py' and './python/reweighted_training.py'.

### Overview
The goal of this projects was to explore data artifacts found in one common dataset used to evaluate LLMs and try some techniques to mitigate their effect. Specifically, this will focus on the task of natural language inference (NLI) and the Stanford NLI (SNLI) dataset. This dataset has been shown to contian multiple artifacts but I will focus on 3 syntactic heuristics (these will be described in the next section). I attempted to use two different techniques to reduce the impact of the dataset artifacts: adding training data, and adapting the training loss function. The two techniques used both seem capable of reducing the impact of these artifacts to varying degrees, but are not without their downsides.  

### NLI and SNLI
NLI is the task of determining if a premise sentence entails (supports), is neutral to, or contradicts a hypothesis sentence. It has been used recently as one way of evaluating if LLMs are able to understand the contents of a sentence. One widely used benchmark dataset for this task is SNLI, which contains 500k+ sentence pairs. An example sentence pair from SNLI is:

premise: A baby at the end of a slip and slide at a party

hypothesis: The baby is wet.

In this example the correct output label is entailment because going down a slip and slide results in getting wet. Most examples in the SNLI dataset are farily easy for humans to reach the correct answer, but can be an issue for less sophisticated machine learning models. 

### SNLI Artifacts
SNLI contains many artifacts. One simple example is that hypotheses containing the word 'not' correspond to output labels of contradiction with a probability much more than random chance. This is highly problematic because any sophisticated LLM can easily pick up on basic features like these and use them to classify sentence pairs. This allows LLMs to acheive high performance without actually preforming the intended and much more challenging task of understanding the sentences to see if they support each other. 

The specifc artifact this project focuses on are all to do with...

### HANS Dataset


### Base Model

### Improving the Base Model

#### Adding Training Data

#### Reweighting Observations

### Results